##**Pose Estimation with ICP**

## Notebook Setup 
The following cell will install Drake, checkout the manipulation repository, and set up the path (only if necessary).
- On Google's Colaboratory, this **will take approximately two minutes** on the first time it runs (to provision the machine), but should only need to reinstall once every 12 hours.  

More details are available [here](http://manipulation.mit.edu/drake.html).

In [ ]:
import importlib
import sys
from urllib.request import urlretrieve

if 'google.colab' in sys.modules and importlib.util.find_spec('manipulation') is None:
    urlretrieve(f"http://manipulation.csail.mit.edu/scripts/setup/setup_manipulation_colab.py",
                "setup_manipulation_colab.py")
    from setup_manipulation_colab import setup_manipulation
    setup_manipulation(manipulation_sha='9099a8e3fcc5c48fe46f4374b3c6be46c11f2d90', drake_version='20200921', drake_build='nightly')

# Determine if this notebook is currently running as a notebook or a unit test.
from IPython import get_ipython
running_as_notebook = get_ipython() and hasattr(get_ipython(), 'kernel')

# Setup rendering (with xvfb), if necessary:
import os
if 'google.colab' in sys.modules and os.getenv("DISPLAY") is None:
    from pyvirtualdisplay import Display
    display = Display(visible=0, size=(1400, 900))
    display.start()

# Start a single meshcat server instance to use for the remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
server_args = []
if 'google.colab' in sys.modules:
    server_args = ['--ngrok_http_tunnel']
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=server_args)

import numpy as np
import meshcat
import open3d as o3d
import meshcat.geometry as g
import meshcat.transformations as tf

from pydrake.all import (
    AddMultibodyPlantSceneGraph, BaseField, Box, ConnectMeshcatVisualizer, DepthCameraProperties, 
    DepthImageToPointCloud, DiagramBuilder, FindResourceOrThrow, MakeRenderEngineVtk, CameraInfo, PixelType, 
    MeshcatPointCloudVisualizer, Parser, RenderEngineVtkParams, RgbdSensor, RigidTransform, Simulator,
    RollPitchYaw, RotationMatrix, SpatialInertia, LeafSystem, AbstractValue, PointCloud, Fields)

## Problem Description
Last lecture, we designed pick and place trajectories **assuming** that the object pose ${}^W X^O$ was known. With all the tools we have learned for goemetric perception, it is time to relax this assumption and finally do pose estimation from sensor data. 

The goal of the exercise is to give you some real-world experience into what dealing with depth cameras, and what it takes to go from a real depth image to the clean ICP formulation we learned.

**These are the main steps of the exercise:**
1. Perform Segmentation on the raw pointcloud of the scene to extract pointcloud from the object.
2. Tune an off-the-shelf ICP solver and estimate the pose of the object.

Before jumping into the main exercise, how should we computationally represent a pointcloud? If we say that pointcloud has $N$ points, then each point has a position in 3D, ${}^Cp^i$, as well as an associated color. Throughout this exercise, we will tend to store them as separate arrays of:
- `Nx3` numpy array where each row stores the XYZ position of the point in meters.
- `Nx3` numpy array where each row stores the RGB information of the point in `uint8` format. 

But many opensource libraries have their own implementations of the `PointCloud` class, mainly because they have highly optimized the underlying operations that take place under the hood, usually at the `C/C++`-level. [Drake](https://drake.mit.edu/) has a [PointCloud](https://drake.mit.edu/pydrake/pydrake.perception.html#pydrake.perception.PointCloud) class, and [Open3d](http://www.open3d.org/) has its own [PointCloud](http://www.open3d.org/docs/release/python_api/open3d.geometry.PointCloud.html) class. We will need to switch between different representations, so below here are some helper functions to get us started. 

In [ ]:
"""
Point Cloud Utilities. Converts between three forms of pointcloud objects:
  - Drake Pointcloud Object
  - Open3d Pointcloud Object
  - Nx3 numpy array
"""

def pcl_np2o3d(pcl_np):
  """
  Input: Nx3 np array of points.
  Ouput: Open3d np object. 
  """
  assert(pcl_np.shape[1] == 3) # sanity check
  pcl_o3d = o3d.geometry.PointCloud()
  pcl_o3d.points = o3d.utility.Vector3dVector(pcl_np)
  return pcl_o3d 

def pcl_o3d2np(pcl_o3d):
  """
  Input: Open3d np object.
  Output: Nx3 np array of points. 
  """
  return np.asarray(pcl_o3d.points)
  
def pcl_np2drake(pcl_np, color):
  """
  Input: Nx3 np array of points in mm, and and 3x1 np array of color using uin8 format.
  Output: drake Pointcloud object. 
  """
  assert(pcl_np.shape[1] == 3) # sanity check. 
  pcl_drake = PointCloud(new_size = pcl_np.shape[0],
                         fields= Fields(BaseField.kXYZs | BaseField.kRGBs))
  xyzs = pcl_drake.mutable_xyzs()
  xyzs[:,:] = np.array(pcl_np).transpose()
  rgbs = pcl_drake.mutable_rgbs()
  rgbs[:,:] = np.tile(color, (pcl_np.shape[0], 1)).transpose()
  return pcl_drake

def pcl_drake2np(pcl_drake):
  """
  Input: drake Pointcloud object.
  Output: Nx3 np array of points. 
  """
  xyzs = pcl_drake.mutable_xyzs().copy().transpose()
  return xyzs 

def rgb_drake2np(pcl_drake):
  rgbs = pcl_drake.mutable_rgbs().copy().transpose()
  return rgbs 

def visualize_pointcloud(vis, pcl, name="point_cloud", X_WC=RigidTransform()):
  """
  pcl: Drake pointcloud object. 
  All other formats should convert to this format before passing to visualization. 
  """
  vis[name].set_object(g.PointCloud(X_WC.multiply(pcl.xyzs()), pcl.rgbs() / 255., size=0.001))

def visualize_pointcloud_xyzrgb(vis, pcl_xyz, pcl_rgb, name="point_cloud", X_WC=RigidTransform()):
  """
  Visualization function to access Meshcat's method more directly. 
  """
  vis[name].set_object(g.PointCloud(X_WC.multiply(pcl_xyz), pcl_rgb / 255., size=0.001))

## Getting a Pointcloud of the Model ##

Before taking a pointcloud of the **scene**, we will need a pointcloud of the **model** to compare against. Generally, this can be done by using existing tools that convert 3D representations (meshes, signed distance functions, etc.) into pointclouds.  

Since our red foam brick is of rectangular shape, we'll cheat a bit and generate the points procedurally. When you click the cell below, you should be able to see the red brick and our pointcloud representation of the brick as blue dots. 

We will save the model pointcloud in the variable `model_pcl_np`. 

In [ ]:
def visualize_red_foam_brick():
  """
  Visualize red foam brick in Meshcat.
  """
  builder = DiagramBuilder()
  plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)
  parser = Parser(plant)
  parser.AddModelFromFile(FindResourceOrThrow(
        "drake/examples/manipulation_station/models/061_foam_brick.sdf"))
  plant.Finalize()

  # Setup Meshcat
  frames_to_draw = {"foam_brick": "base_link"}
  meshcat = ConnectMeshcatVisualizer(builder, scene_graph, 
                                     zmq_url=zmq_url, 
                                     delete_prefix_on_load=False,
                                     frames_to_draw=frames_to_draw)
  meshcat.load()
  builder.Build()
  return meshcat.vis

def generate_model_pointcloud(xrange, yrange, zrange, res):
  """
  Procedurally generate pointcloud of a rectangle for each side. 
  """
  # Decide on how many samples 
  x_lst = np.linspace(xrange[0], xrange[1], int((xrange[1] - xrange[0]) / res))
  y_lst = np.linspace(yrange[0], yrange[1], int((yrange[1] - yrange[0]) / res))
  z_lst = np.linspace(zrange[0], zrange[1], int((zrange[1] - zrange[0]) / res))

  pcl_lst = [] 
  # Do XY Plane 
  for x in x_lst:
    for y in y_lst:
      pcl_lst.append([x, y, zrange[0]])
      pcl_lst.append([x, y, zrange[1]])

  # Do YZ Plane
  for y in y_lst:
    for z in z_lst:
      pcl_lst.append([xrange[0], y, z])
      pcl_lst.append([xrange[1], y, z])

  # Do XZ Plane
  for x in x_lst:
    for z in z_lst:
      pcl_lst.append([x, yrange[0], z])
      pcl_lst.append([x, yrange[1], z])


  return np.array(pcl_lst)

vis = visualize_red_foam_brick()
model_pcl_np = generate_model_pointcloud([-0.0375, 0.0375], [-0.025, 0.025], [0., 0.05], 0.002)
visualize_pointcloud(vis, pcl_np2drake(model_pcl_np, [0, 0, 255]), "pcl_scene")

## Getting the Scene Pointcloud## 

Now let's set up the ClutteringStation from last lecture and actually take a pointcloud snapshot of the scene with the `red_foam_brick`. We'll place the camera where we have good coverage of the bin. We'll also take a pointcloud snapshot without the `red_foam_brick` so that we can use it for segmentation later.

NOTE: There are around `3e7` points that are trying to be published to the visualizer, so things might load slowly, and occasionally the Colab session might crash. Keep calm and run the cells from the beginning! 

In [ ]:
vis.delete()

def setup_clutter_station(X_WO, X_WC):
  builder = DiagramBuilder()
  
  plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)
  parser = Parser(plant)

  # Add the foam brick. 
  if (X_WO is not None):
    brick = parser.AddModelFromFile(FindResourceOrThrow(
        "drake/examples/manipulation_station/models/061_foam_brick.sdf"))
    plant.WeldFrames(plant.world_frame(),
                    plant.GetFrameByName("base_link", brick),
                    X_WO)
  
  bin1 = parser.AddModelFromFile(FindResourceOrThrow(
      "drake/examples/manipulation_station/models/bin.sdf"), "bin1")
  plant.WeldFrames(plant.world_frame(),
                   plant.GetFrameByName("bin_base", bin1),
                   RigidTransform(RollPitchYaw(0, 0, np.pi/2), [-0.145, -0.63, 0.075]))

  bin2 = parser.AddModelFromFile(FindResourceOrThrow(
      "drake/examples/manipulation_station/models/bin.sdf"), "bin2")
  plant.WeldFrames(plant.world_frame(),
                   plant.GetFrameByName("bin_base", bin2),
                   RigidTransform(RollPitchYaw(0, 0, np.pi), [0.5, -0.1, 0.075]))

  # Add a rendering engine
  renderer = "my_renderer"
  scene_graph.AddRenderer(renderer,
                          MakeRenderEngineVtk(RenderEngineVtkParams()))
  
  # Add a mock camera model  
  camera_instance = plant.AddModelInstance("cameras")
  camera = plant.AddRigidBody("camera", camera_instance, SpatialInertia())
  plant.RegisterVisualGeometry(
      camera, RigidTransform([0, 0, -0.01]), Box(width=.1, depth=.02, height=.02), "D415", 
      [.4, .4, .4, 1.])
  plant.WeldFrames(plant.world_frame(), camera.body_frame(), X_WC) 
  plant.Finalize() 

  frames_to_draw = {"cameras": "camera"}
  meshcat = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url, 
                                      delete_prefix_on_load=False,
                                      frames_to_draw=frames_to_draw)
  meshcat.load() 

  properties = DepthCameraProperties(width=640,
                                      height=480,
                                      fov_y=np.pi / 4.0,
                                      renderer_name=renderer,
                                      z_near=0.1,
                                      z_far=10.0)
  camera = builder.AddSystem(
      RgbdSensor(parent_id=plant.GetBodyFrameIdOrThrow(camera.index()),
                  X_PB=RigidTransform(),
                  properties=properties,
                  show_window=False))
  camera.set_name("rgbd_sensor")
  builder.Connect(scene_graph.get_query_output_port(),
                  camera.query_object_input_port())

  # Export the camera outputs
  builder.ExportOutput(camera.color_image_output_port(), "color_image")
  builder.ExportOutput(camera.depth_image_32F_output_port(), "depth_image")

  # Add a system to convert the camera output into a point cloud
  to_point_cloud = builder.AddSystem(
      DepthImageToPointCloud(camera_info=camera.depth_camera_info(),
                              pixel_type=PixelType.kDepth32F,
                              fields=BaseField.kXYZs | BaseField.kRGBs))
  builder.Connect(camera.depth_image_32F_output_port(),
                  to_point_cloud.depth_image_input_port())
  builder.Connect(camera.color_image_output_port(),
                  to_point_cloud.color_image_input_port())

  # Send the point cloud to meshcat for visualization, too.
  meshcat_pointcloud = builder.AddSystem(MeshcatPointCloudVisualizer(meshcat, X_WP=X_WC, draw_period=1./5.))
  builder.Connect(to_point_cloud.point_cloud_output_port(), meshcat_pointcloud.get_input_port())

  # Export the point cloud output.
  builder.ExportOutput(to_point_cloud.point_cloud_output_port(),
                        "point_cloud")

  diagram = builder.Build()
  diagram.set_name("depth_camera_demo_system")
  return diagram 

# Set pose of the brick 
X_WO = RigidTransform(RollPitchYaw(0, 0, np.pi/5).ToRotationMatrix(), np.array([-0.1, -0.6, 0.09]))

# Setup CameraPose
X_WC = RigidTransform(
    RollPitchYaw(0, 0, 0).ToRotationMatrix().multiply(
        RollPitchYaw(-np.pi/2. - np.pi/3, 0, 0).ToRotationMatrix()),
    [-.1, -.8, .5])

# Take a pointcloud snapshot of the background to use for subtraction 
diagram = setup_clutter_station(None, X_WC)
simulator = Simulator(diagram)
simulator.AdvanceTo(0.01)
context = simulator.get_context()
scene_pcl_drake_background =  diagram.GetOutputPort("point_cloud").Eval(context)
scene_pcl_np_background = X_WC.multiply(pcl_drake2np(scene_pcl_drake_background).transpose()).transpose()
scene_rgb_np_background = rgb_drake2np(scene_pcl_drake_background)

# Take a pointcloud snapshot of the scene with the brick. 
diagram = setup_clutter_station(X_WO, X_WC)
simulator = Simulator(diagram)
simulator.AdvanceTo(0.01)
context = simulator.get_context()
scene_pcl_drake = diagram.GetOutputPort("point_cloud").Eval(context)
scene_pcl_np = X_WC.multiply(pcl_drake2np(scene_pcl_drake).transpose()).transpose()
scene_rgb_np = rgb_drake2np(scene_pcl_drake)

## Visualizing the Problem ##

That was a lot of work, but if you run the below cell, Meshcat will finally show you a clean formulation of the main problem. We have 3 pointcloud objects in Meshcat:

- `pcl_model`: Pointcloud of models
- `pcl_scene`: Raw pointcloud of the foam-brick scene obtained from a RGBD camera.
- `pcl_scene_background`: Raw pointcloud of the background obtained from a RGBD camera. 

In case you forgot, In Meshcat's menu you can go into the `meshcat` tab and turn different objects on and off so that you can see what the background pointcloud looks like as well. 

NOTE: You might have to wait a bit until the bin pointcloud shows up.



In [ ]:
vis = meshcat.Visualizer(zmq_url)
vis.delete()

visualize_pointcloud(vis, pcl_np2drake(model_pcl_np, [0, 0, 255]), name="pcl_model")
visualize_pointcloud(vis, scene_pcl_drake, name="pcl_scene", X_WC=X_WC)
visualize_pointcloud_xyzrgb(vis, 
                            scene_pcl_np_background.transpose(), 
                            scene_rgb_np_background.transpose(), 
                            name="pcl_scene_background")

If we simply run ICP with `pcl_model` and `pcl_scene`, we might get a terrible result because there might be features in the background that the model is trying to run correspondence with. So we'd like to vet the problem a bit and perform **segmentation**: which parts of the scene pointcloud corresponds to an actual point on the `red_foam_brick`? 


**Now it's your turn to code!**

Below, you will implement a function `segment_scene_pcl` that takes in a pointcloud of the scene and return the relevant points that are actually on the `red_foam_brick`. But here are the rules of the game:
- You **may** use color data, the background pointcloud, and any outlier detection algorithm that you can write to perform segmentation.
- You may **not** explicitly impose conditions on the position to filter out the data. Remember that our goal is to estimate the pose in the first place, so using position will be considered cheating.
- You may **not** use external libraries that are not in this notebook already. 

In order to get full score for this assignment, you need to satisfy both criteria:
- The number of false outliers (points which are not on the red brick but was caught by segmentation) must not exceed 80 points.
- The number of missed inliers (points that are on the red brick but was not caught by segmentation) must not exceed 80 points. 

You will be able to visualize your segmented pointclouds on Meshcat by running the cell.

In [ ]:
def segment_scene_pcl(scene_pcl_np, scene_rgb_np, scene_pcl_np_background, scene_rgb_np_background):
  """
  Inputs: 
  scene_pcl_np: Nx3 np.float32 array of pointclouds, each row containing xyz position of each point in meters. 
  scene_rgb_np: Nx3 np.uint8   array of pointclouds, each row containing rgb color data of each point.
  scene_pcl_np_background: Nx3 np.float32 array of pointclouds, each row containing xyz position of each point in meters. 
  scene_rgb_np_background: Nx3 np.uint8   array of pointclouds, each row containing rgb color data of each point.

  Outputs:
  scene_pcl_np_filtered: Mx3 np.float32 array of pointclouds that are on the foam brick. 
  """
  ####################
  # Fill your code here. 

  scene_pcl_np_filtered = scene_pcl_np
  ####################
 
  return scene_pcl_np_filtered

scene_pcl_np_filtered = segment_scene_pcl(scene_pcl_np,
                                          scene_rgb_np,
                                          scene_pcl_np_background,
                                          scene_rgb_np_background)
visualize_pointcloud(vis, pcl_np2drake(scene_pcl_np_filtered, [0, 255, 0]), 
                     name='pcl_scene_filtered')

## ICP for Pose Estimation

Now that we have a subset of scene points that we want to use to estimate the pose, let's do ICP to figure out what ${}^W X^O$ is. Instead of implementing your own ICP this time, we will use the most common implementation available: it comes from [Open3d](http://www.open3d.org/) and implements the method from [[Rusinkiewicz, 2001]](http://www.pcl-users.org/file/n4037867/Rusinkiewicz_Effcient_Variants_of_ICP.pdf).

We know that ICP can't work very well without even a rough initialization. Let's assume that we at least know that the `red_foam_brick` is inside the bin, so that we can initialize the ${}^W X^O$ to be at the center of the bin with an identity rotation. 

In [ ]:
initial_guess = RigidTransform()
initial_guess.set_translation([-0.145, -0.63, 0.09]) # do not modify 

def o3d_icp(model_pcl_np, scene_pcl_np_filtered, initial_guess, solver_params): 
  """Performs ICP with Open3d's Generalized-ICP implementation.
  Input: 
    - model_pcl_np: Nx3 np.float32 array, model pointcloud 
    - scene_pcl_np_filtered: Nx3 np.float32 array, model pointcloud
    - initial_guess: Drake RigidTransform object used as initial guess to correspondence. 
    - solver_params: dictionary containing solver parameters.
  Output:
    - X_MS: Estimated relative transform between model and scene pointclouds
  """
  source = pcl_np2o3d(model_pcl_np)
  target = pcl_np2o3d(scene_pcl_np_filtered)

  reg_p2p = o3d.registration.registration_icp(
      source, target, 
      solver_params["max_correspondence_distance"], initial_guess.GetAsMatrix4(),
      estimation_method = o3d.registration.TransformationEstimationPointToPoint(),
      criteria = o3d.registration.ICPConvergenceCriteria(
          relative_fitness=solver_params["relative_fitness"],
          relative_rmse=solver_params["relative_rmse"],
          max_iteration=solver_params["max_iteration"])
  )

  X_MS_hat = RigidTransform(reg_p2p.transformation)

  return X_MS_hat

You will notice that Open3d's ICP method exposes some parameters you need to tune for it to be working successfully. The [documentation](http://www.open3d.org/docs/release/python_api/open3d.registration.registration_icp.html?highlight=registration_icp) describes the parameters as follows: 

- `max_correspondence_distance`: maximum distance for pair of points to be registered as a correspondence. 
- `relative_fitness`: If relative change (difference) of fitness score is lower than `relative_fitness`, the iteration stops. 
- `relative_rmse`: If relative change (difference) of inlier RMSE score is lower than `relative_rmse`, the iteration stops. 
- `max_iteration`: maximum iteration before iteration stops.

**Now, it's your time to code**

The problem is quite straightforward - if you had a good segmented pointcloud for the previous question, can you tweak the solver to find a good estimate of ${^WX^O}$? 

You should be able to check the validity of your solution on Meshcat by running the cell below.

Your estimated transform is stored in a variable `X_MS_hat`, which we will be used for the grading code. 


In [ ]:
# Modify this dictionary. 
solver_params = {
    "max_correspondence_distance": 1.0,
    "relative_fitness": 1e-6,
    "relative_rmse": 1e-6,
    "max_iteration": 30
}

X_MS_hat = o3d_icp(model_pcl_np, scene_pcl_np_filtered, initial_guess, solver_params)
visualize_pointcloud(vis, pcl_np2drake(model_pcl_np, [255, 0, 255]), name="pcl_estimated", X_WC=X_MS_hat)

In order to get full points, the relative transform ${}^OX^{\hat{O}}$, between the true pose ${}^W X^O$ and estimated pose ${}^WX^\hat{O}$, must satisfy the following criteria: 
- xyz translation values of ${}^O X^{\hat{O}}$ does not exceed 0.005 (5 milimeters)
- RollPitchYaw angles of ${}^O X^{\hat{O}}$ does not exceed 0.05 (about 3degs)

We will give partial scores to translations of less than 0.02 (2 centimeters), and rotations less than 0.05 (~30 degrees)

In [ ]:
np.set_printoptions(precision=3, suppress=True)
X_OOhat = X_MS_hat.inverse().multiply(X_WO)

rpy = RollPitchYaw(X_OOhat.rotation()).vector()
xyz = X_OOhat.translation()

print("RPY Value: " + str(rpy))
print("XYZ Value: " + str(xyz))

##How will this notebook be Graded?##

If you are enrolled in the class, this notebook will be graded using [Gradescope](www.gradescope.com). You should have gotten the enrollement code on our announcement in Piazza. 

For submission of this assignment, you must do as follows:. 
- Download and submit the notebook `pick_and_place_perception.ipynb` to Gradescope's notebook submission section, along with your notebook for the other problems.

We will evaluate the local functions in the notebook to see if the function behaves as we have expected. For this exercise, the rubric is as follows:
- [4 pts] `segment_scene_pcl` correctly segments the scene by having less than 80 missed inliers and 80 false outliers.
- [1 pts] XYZ Translation of ${}^O X^{\hat{O}}$ does not exceed 0.02. 
- [1 pts] XYZ Translation of ${}^O X^{\hat{O}}$ does not exceed 0.005. 
- [1 pts] RollPitchYaw angles of ${}^O X^{\hat{O}}$ does not exceed 0.5. 
- [1 pts] RollPitchYaw angles of ${}^O X^{\hat{O}}$ does not exceed 0.05. 

Unlike other exercises, there are partial points for this exercise depending on how much you've violated these requirements. 

Below is our autograder where you can check your score!

In [ ]:
from manipulation.exercises.pose.test_pose_estimation import TestPoseEstimation
from manipulation.exercises.grader import Grader 

Grader.grade_output([TestPoseEstimation], [locals()], 'results.json')
Grader.print_test_results('results.json')